# Notebook 4: RAG/Retrieval Context Quality for Policy Documents

**Objective**: Ensure approval decisions use relevant, current policy documents:
- Detect out-of-distribution queries (novel request types)
- Evaluate retrieval context quality
- Handle irrelevant or outdated policy matches

---

## Flow Diagram

```mermaid
flowchart TD
    subgraph Input["📥 Approval Request"]
        A[Request Text]
        B[Request Embedding]
    end

    subgraph RAG["📚 Policy Retrieval"]
        C[Policy Document Store]
        D[Semantic Search]
        E[Top-K Retrieved Chunks]
        F[Retrieval Scores]
    end

    subgraph Cleanlab["🧹 Cleanlab OOD Analysis"]
        G[OutOfDistribution.fit]
        H[OOD Score per Query]
        I[find_overlapping_classes]
        J[Policy Category Confusion]
    end

    subgraph Quality["🔍 Context Quality"]
        K{Query In-Distribution?}
        L[High OOD → Novel Query → N High]
        M[Low Retrieval Score → Irrelevant → S High]
        N[Category Confusion → N Moderate]
        O[Good Match → R High]
    end

    subgraph YRSN["🎯 YRSN Decomposition"]
        P[Combine Signals]
        Q[CONFUSION Collapse Check]
    end

    subgraph Routing["🚦 Temperature Routing"]
        R[Compute τ = 1/α]
        S{Context Quality?}
        T[🟢 GREEN: Trust context]
        U[🟡 YELLOW: Human verify context]
        V[🔴 RED: Manual policy lookup]
    end

    A --> B
    B --> D
    C --> D
    D --> E
    D --> F
    B --> G
    G --> H
    E --> I
    I --> J
    H --> K
    K -->|OOD| L
    F --> K
    K -->|Low score| M
    J --> N
    K -->|Good| O
    L --> P
    M --> P
    N --> P
    O --> P
    P --> Q
    Q --> R
    R --> S
    S -->|High R| T
    S -->|Medium R| U
    S -->|Low R| V

    style RAG fill:#e1f5fe
    style Cleanlab fill:#fff3e0
    style Quality fill:#e8f5e9
    style YRSN fill:#fce4ec
```

---

**Collapse Type Focus**: CONFUSION (wrong policy retrieved)

**Difficulty**: ⭐⭐ Medium

## 1. Setup

In [ ]:
# Install dependencies
!pip install cleanlab sentence-transformers scikit-learn pandas faiss-cpu --quiet

In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss

# Cleanlab for OOD detection
from cleanlab.outlier import OutOfDistribution

# Import YRSN adapter
import sys
sys.path.append('../src')
from yrsn_iars.adapters.cleanlab_adapter import CleanlabAdapter, YRSNResult, CollapseType
from yrsn_iars.adapters.temperature import compute_temperature

print("Dependencies loaded successfully")

## 2. Create Policy Document Store

Simulate a knowledge base of corporate policy documents.

In [ ]:
# Policy document corpus
policy_documents = [
    # Travel Policy
    {"category": "travel", "doc": "travel_policy_v2.pdf", 
     "text": "Travel expenses must be pre-approved for amounts exceeding $500. Business class flights require VP approval for trips over 6 hours. Hotel expenses capped at $250/night."},
    {"category": "travel", "doc": "travel_policy_v2.pdf",
     "text": "International travel requires 2-week advance notice and security review. Per diem rates follow GSA guidelines."},
    
    # Expense Policy
    {"category": "expense", "doc": "expense_policy_v3.pdf",
     "text": "Team meals and entertainment expenses require itemized receipts. Maximum $100 per person for client dinners."},
    {"category": "expense", "doc": "expense_policy_v3.pdf",
     "text": "Home office equipment up to $500 can be expensed once per year. Must be job-related."},
    
    # Software/License Policy
    {"category": "software", "doc": "software_procurement.pdf",
     "text": "Software licenses must be procured through IT. Cloud services require security review. Annual license renewals auto-approved if under $10,000."},
    {"category": "software", "doc": "software_procurement.pdf",
     "text": "Open source software usage requires legal review. No unapproved SaaS tools for customer data."},
    
    # Vendor Policy
    {"category": "vendor", "doc": "vendor_management.pdf",
     "text": "New vendors require due diligence review. Contracts over $50,000 need legal review. Preferred vendor list maintained by procurement."},
    {"category": "vendor", "doc": "vendor_management.pdf",
     "text": "Vendor payments net-30 unless negotiated. Early payment discounts require CFO approval."},
    
    # Equipment Policy
    {"category": "equipment", "doc": "asset_management.pdf",
     "text": "IT equipment refresh cycle is 3 years for laptops. Requests for non-standard equipment require manager justification."},
    {"category": "equipment", "doc": "asset_management.pdf",
     "text": "Equipment disposal must follow data destruction protocols. Returns processed within 30 days of employee departure."},
]

policies_df = pd.DataFrame(policy_documents)
print(f"Policy corpus: {len(policies_df)} chunks from {policies_df['doc'].nunique()} documents")
print(f"Categories: {policies_df['category'].unique()}")

In [ ]:
# Create embeddings for policy documents
encoder = SentenceTransformer('all-MiniLM-L6-v2')

print("Embedding policy documents...")
policy_embeddings = encoder.encode(policies_df['text'].tolist(), show_progress_bar=True)

print(f"Policy embeddings shape: {policy_embeddings.shape}")

In [ ]:
# Build FAISS index for retrieval
dimension = policy_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner product (cosine sim for normalized vectors)

# Normalize for cosine similarity
faiss.normalize_L2(policy_embeddings)
index.add(policy_embeddings)

print(f"FAISS index built with {index.ntotal} documents")

## 3. Generate Approval Requests (Including OOD)

In [ ]:
# Mix of in-distribution and OOD requests
requests = [
    # In-distribution (should match policies)
    {"id": "REQ-001", "text": "Can I expense a team lunch for 5 people at $80 per person?", "expected_category": "expense"},
    {"id": "REQ-002", "text": "Need approval for AWS license renewal, annual cost $8,000", "expected_category": "software"},
    {"id": "REQ-003", "text": "Requesting business class flight to London (8 hour flight)", "expected_category": "travel"},
    {"id": "REQ-004", "text": "New vendor onboarding for consulting services, $75,000 contract", "expected_category": "vendor"},
    {"id": "REQ-005", "text": "Request new laptop replacement, current one is 4 years old", "expected_category": "equipment"},
    {"id": "REQ-006", "text": "Per diem for 5-day conference in Chicago", "expected_category": "travel"},
    {"id": "REQ-007", "text": "Home office monitor and keyboard expense, $450 total", "expected_category": "expense"},
    {"id": "REQ-008", "text": "Salesforce license for new sales team member", "expected_category": "software"},
    
    # Out-of-distribution (novel request types)
    {"id": "REQ-009", "text": "Request for cryptocurrency mining hardware for R&D project", "expected_category": "OOD"},
    {"id": "REQ-010", "text": "Need approval for employee wellness retreat in Bali", "expected_category": "OOD"},
    {"id": "REQ-011", "text": "Requesting company sponsorship for my kid's little league team", "expected_category": "OOD"},
    {"id": "REQ-012", "text": "Can we hire a DJ for the office holiday party?", "expected_category": "OOD"},
    
    # Ambiguous (could match multiple policies)
    {"id": "REQ-013", "text": "Need both software and travel for remote conference with cloud vendor", "expected_category": "ambiguous"},
    {"id": "REQ-014", "text": "Equipment purchase from new vendor, need expedited payment", "expected_category": "ambiguous"},
]

requests_df = pd.DataFrame(requests)
print(f"Generated {len(requests_df)} test requests")
print(f"\nCategory distribution:")
print(requests_df['expected_category'].value_counts())

## 4. Perform Retrieval and OOD Detection

In [ ]:
# Embed requests
request_embeddings = encoder.encode(requests_df['text'].tolist())
faiss.normalize_L2(request_embeddings)

# Retrieve top-3 policy chunks for each request
k = 3
distances, indices = index.search(request_embeddings, k)

# Store retrieval results
retrieval_results = []
for i, (req_id, dists, idxs) in enumerate(zip(requests_df['id'], distances, indices)):
    for rank, (dist, idx) in enumerate(zip(dists, idxs)):
        retrieval_results.append({
            'request_id': req_id,
            'rank': rank + 1,
            'policy_idx': idx,
            'policy_category': policies_df.iloc[idx]['category'],
            'policy_doc': policies_df.iloc[idx]['doc'],
            'similarity': dist
        })

retrieval_df = pd.DataFrame(retrieval_results)
print("Top retrieval results:")
retrieval_df.head(9)

In [ ]:
# Cleanlab OOD Detection
# Train OOD detector on policy embeddings
ood = OutOfDistribution()
ood.fit(policy_embeddings)

# Score each request
ood_scores = ood.score(request_embeddings)

# Note: Cleanlab OOD score: higher = more likely OOD
# We invert for YRSN: higher = more in-distribution
requests_df['ood_score_raw'] = ood_scores
requests_df['in_distribution_score'] = 1 - ood_scores

print("OOD Scores:")
print(requests_df[['id', 'expected_category', 'ood_score_raw', 'in_distribution_score']].to_string())

## 5. Compute Context Quality Signals

In [ ]:
# Aggregate retrieval quality per request
def compute_retrieval_quality(request_id):
    """Compute retrieval quality metrics for a request."""
    req_results = retrieval_df[retrieval_df['request_id'] == request_id]
    
    # Top-1 similarity
    top1_sim = req_results.iloc[0]['similarity']
    
    # Mean similarity of top-3
    mean_sim = req_results['similarity'].mean()
    
    # Category consistency (do retrieved chunks agree?)
    categories = req_results['policy_category'].tolist()
    category_agreement = len(set(categories)) == 1
    
    # Similarity gap (top-1 vs top-2)
    sim_gap = req_results.iloc[0]['similarity'] - req_results.iloc[1]['similarity']
    
    return {
        'top1_similarity': top1_sim,
        'mean_similarity': mean_sim,
        'category_agreement': category_agreement,
        'similarity_gap': sim_gap,
        'top_category': categories[0]
    }

# Apply to all requests
quality_metrics = requests_df['id'].apply(compute_retrieval_quality).apply(pd.Series)
requests_df = pd.concat([requests_df, quality_metrics], axis=1)

print("Retrieval Quality Metrics:")
requests_df[['id', 'expected_category', 'top1_similarity', 'category_agreement', 'top_category']]

## 6. YRSN Decomposition for RAG Context

In [ ]:
adapter = CleanlabAdapter()

def compute_rag_yrsn(row):
    """Compute YRSN specifically for RAG context quality."""
    
    # N (Noise): OOD queries and category confusion
    n_ood = row['ood_score_raw']  # Higher = more OOD = more N
    n_confusion = 0.2 if not row['category_agreement'] else 0.0
    N = min(1.0, n_ood + n_confusion)
    
    # S (Superfluous): Low similarity (irrelevant context)
    # If similarity is low, context is not useful
    s_irrelevant = max(0, 0.7 - row['top1_similarity'])  # Below 0.7 = irrelevant
    s_ambiguous = max(0, 0.15 - row['similarity_gap'])   # Small gap = ambiguous
    S = min(1.0 - N, s_irrelevant + s_ambiguous)
    
    # R (Relevant): Good context quality
    R = max(0, 1.0 - N - S)
    
    # Normalize
    total = R + S + N
    return YRSNResult(R=R/total, S=S/total, N=N/total)

# Apply YRSN computation
yrsn_results = requests_df.apply(compute_rag_yrsn, axis=1)
requests_df['R'] = [y.R for y in yrsn_results]
requests_df['S'] = [y.S for y in yrsn_results]
requests_df['N'] = [y.N for y in yrsn_results]
requests_df['collapse_type'] = [y.collapse_type.value for y in yrsn_results]

print("YRSN for RAG Context:")
requests_df[['id', 'expected_category', 'R', 'S', 'N', 'collapse_type']]

## 7. Temperature-Based Routing for RAG

In [ ]:
# Compute temperature
requests_df['temperature'] = requests_df['R'].apply(lambda r: compute_temperature(r))

# RAG-specific routing
def route_rag_request(row):
    tau = row['temperature']
    collapse = row['collapse_type']
    
    # CONFUSION collapse: force red (wrong policy retrieved)
    if collapse == 'confusion':
        return 'red'
    
    # High temperature (low R): need human to verify/find policy
    if tau > 2.0:
        return 'red'
    
    # Medium temperature: human reviews retrieved context
    elif tau > 1.3:
        return 'yellow'
    
    # Low temperature: trust the context
    else:
        return 'green'

requests_df['stream'] = requests_df.apply(route_rag_request, axis=1)

print("RAG Routing Results:")
print(requests_df[['id', 'expected_category', 'R', 'temperature', 'stream', 'collapse_type']])

print(f"\n" + "="*50)
print(f"Routing Distribution:")
print(requests_df['stream'].value_counts())

## 8. Analyze OOD and CONFUSION Cases

In [ ]:
# OOD Cases (novel requests)
ood_cases = requests_df[requests_df['expected_category'] == 'OOD']

print("OUT-OF-DISTRIBUTION Cases:")
print("="*60)
for _, row in ood_cases.iterrows():
    print(f"\n[{row['id']}] {row['text'][:60]}...")
    print(f"  OOD Score: {row['ood_score_raw']:.3f} (higher = more OOD)")
    print(f"  Top Match: {row['top_category']} (sim={row['top1_similarity']:.3f})")
    print(f"  YRSN: R={row['R']:.2f}, S={row['S']:.2f}, N={row['N']:.2f}")
    print(f"  Stream: {row['stream'].upper()}, τ={row['temperature']:.2f}")

In [ ]:
# CONFUSION Cases (category disagreement)
confusion_cases = requests_df[~requests_df['category_agreement']]

print("CONFUSION Cases (Category Disagreement in Retrieved Docs):")
print("="*60)
for _, row in confusion_cases.iterrows():
    req_results = retrieval_df[retrieval_df['request_id'] == row['id']]
    categories = req_results['policy_category'].tolist()
    
    print(f"\n[{row['id']}] {row['text'][:60]}...")
    print(f"  Retrieved categories: {categories}")
    print(f"  YRSN: R={row['R']:.2f}, S={row['S']:.2f}, N={row['N']:.2f}")
    print(f"  Collapse: {row['collapse_type']}")
    print(f"  Stream: {row['stream'].upper()}")

## 9. Export Results

In [ ]:
# Save results
output_cols = ['id', 'text', 'expected_category', 'top_category',
               'top1_similarity', 'category_agreement', 'in_distribution_score',
               'R', 'S', 'N', 'collapse_type', 'temperature', 'stream']

requests_df[output_cols].to_csv('rag_yrsn_results.csv', index=False)
retrieval_df.to_csv('retrieval_results.csv', index=False)

print(f"Saved RAG analysis results")

## Summary

In this notebook we:
1. Created a policy document knowledge base with semantic search
2. Used Cleanlab OOD detection to identify novel request types
3. Computed retrieval quality metrics (similarity, category agreement)
4. Mapped to YRSN: OOD → N, Low similarity → S, Category confusion → N
5. Applied temperature-based routing for RAG decisions
6. Identified CONFUSION collapse cases (wrong policy retrieved)

**Key Insight**: When retrieved context is uncertain (OOD query or low similarity), temperature increases, routing to yellow/red for human verification of policy applicability.

**Next**: Notebook 5 - Token/NER Classification for Contract Entities